In [ ]:
import glob, sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#import tensorflow as tf
import re
import os
import glob
#from PrepareData import PrepareData
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
#from PrepareData import PrepareData
import math
fft_size = 16777216
time_res = 64e-6 # in seconds
T_obs = (fft_size*time_res)/60 # in minutes is equal to 17.895 minutes
freq_axis = np.fft.rfftfreq(fft_size, d=64e-6)
freq_res = 1/(T_obs*60)

In [2]:
labels_df = pd.read_csv('meta_data/labels_runBB.csv')

In [4]:
cur_dir = ''
run = 'runBB'

In [ ]:
# preparing the data chunks
data_type = "test"
indices = np.load(cur_dir + f'raw_data/{run}/{data_type}_indices.npy')
X = np.load(cur_dir + f'raw_data/{run}/{data_type}_data.npy', mmap_mode='r')
Y = np.zeros((len(indices),1))

X_freq = np.zeros((X.shape[0],400))
h = 1
np.random.seed(42)
for i in range(X.shape[0]):#2):
    ind = indices[i]
    z_ind = h*labels_df['z'][ind]
    period = labels_df['new_period'][ind]
    freq = 1/period
    freq_ind = freq/freq_res

    # choose a number in the interval
    chunk_freq_ind = np.random.ranint(0,400)
    z = np.abs(z_ind) 
    interval = 2*z
    
    start = chunk_freq_ind - z
    end = chunk_freq_ind + z
    if start < 0:
        start = np.abs(start)
        portion_ind = (interval - start)
        portion = portion_ind/interval
        start_chunk = freq_ind - z + start
        end_chunk = start_chunk + 400
    elif end > 400:
        portion_ind = (interval - (end - 400))
        portion = portion_ind/portion
        end_chunk = freq_ind + z - (end - 400)
        start_chunk = end_chunk - 400
    elif (start > 0) and (end < 400):
        portion = 1
        start_chunk = freq_ind - z
        end_chunk = start_chunk + 400
    else:
        print('something gone wrong with index',i,start,end)
        sys.exit(1)

    Y[i,0] = portion

    X_freq[i,:] = X[i,start_chunk:end_chunk]
    fig, ax = plt.subplots(figsize=(10, 6))  # adjust as necessary
    ax.plot(np.arange(0,400,1),X_freq[i,:])
    print(portion)
    # ax.axvline(Y_test[i,0],color='r',label='freq')
    # ax.axvline(Y_test[i,0]+Y_test[i,1],color='g',label='z')
    # ax.axvline(Y_test[i,0]-Y_test[i,1],color='g')
    # plt.savefig(f'raw_data/runBB/pngs/test_data_fundamental_{ind}_p{period*1000:.2f}ms.png', dpi=100)  # adjust dpi as necessary
    plt.show()
    plt.close()

# np.save(cur_dir + f'raw_data/{run}/test_data_variable_portions.npy',X_test_freq)
# np.save(cur_dir + f'raw_data/{run}/test_labels_variable_portions.npy',Y_test)

In [9]:
data_type = "train"
indices = np.load(cur_dir + f'raw_data/{run}/{data_type}_indices.npy')
X = np.load(cur_dir + f'raw_data/{run}/{data_type}_data.npy', mmap_mode='r')
Y = np.zeros((len(indices),1))

X_freq = np.zeros((X.shape[0],400))
h = 1
np.random.seed(42)
for i in range(X.shape[0]):
    ind = indices[i]
    chunk_start = labels_df['chunk_start'][ind]
    chunk_start_new = int(chunk_start + 800) # keep 400 index space from last signal
    chunk_end_new = chunk_start_new + 400
    X_freq[i,:] = X[i,chunk_start_new:chunk_end_new]

    # fig, ax = plt.subplots(figsize=(10, 6))  # adjust as necessary
    # ax.plot(np.arange(0,400,1),X_freq[i,:])
    # plt.show()
    # plt.close()

np.save(cur_dir + f'raw_data/{run}/{data_type}_data_chunk_classifier.npy',X_freq)
np.save(cur_dir + f'raw_data/{run}/{data_type}_labels_chunk_classifier.npy',Y)

In [ ]:
data_type = "val"
indices = np.load(cur_dir + f'raw_data/{run}/{data_type}_indices.npy')
X = np.load(cur_dir + f'raw_data/{run}/{data_type}_data.npy', mmap_mode='r')
Y = np.zeros((len(indices),1))

X_freq = np.zeros((X.shape[0],400))
h = 1
np.random.seed(42)
for i in range(X.shape[0]):
    ind = indices[i]
    chunk_start = labels_df['chunk_start'][ind]
    chunk_start_new = int(chunk_start + 800) # keep 400 index space from last signal
    chunk_end_new = chunk_start_new + 400
    X_freq[i,:] = X[i,chunk_start_new:chunk_end_new]

    fig, ax = plt.subplots(figsize=(10, 6))  # adjust as necessary
    ax.plot(np.arange(0,400,1),X_freq[i,:])
    plt.show()
    plt.close()

np.save(cur_dir + f'raw_data/{run}/{data_type}_data_chunk_classifier.npy',X_freq)
np.save(cur_dir + f'raw_data/{run}/{data_type}_labels_chunk_classifier.npy',Y)

In [23]:
train_chunks_1 = np.load(cur_dir + f'raw_data/{run}/train_data_chunk_classifier.npy')
train_chunks_2 = np.load(cur_dir + f'raw_data/{run}/train_data_chunk.npy')
train_labels_1 = np.load(cur_dir + f'raw_data/{run}/train_labels_chunk_classifier.npy')
train_labels_2 = np.load(cur_dir + f'raw_data/{run}/train_labels_chunk.npy')

# combine training data and shuffle and save as one
train_chunks = np.concatenate((train_chunks_1,train_chunks_2),axis=0)
train_labels = np.concatenate((train_labels_1,np.ones(train_labels_1.shape[0]).reshape(train_labels_1.shape[0],1)),axis=0)

# shuffle
np.random.seed(42)

indices = np.arange(train_chunks.shape[0])
np.random.shuffle(indices)

train_chunks = train_chunks[indices,:]
train_labels = train_labels[indices,:]

np.save(cur_dir + f'raw_data/{run}/train_data_classifier.npy',train_chunks)
np.save(cur_dir + f'raw_data/{run}/train_labels_classifier.npy',train_labels)

In [29]:
test_chunks_1 = np.load(cur_dir + f'raw_data/{run}/test_data_chunk_classifier.npy')
test_chunks_2 = np.load(cur_dir + f'raw_data/{run}/test_data_chunk.npy')
test_labels_1 = np.load(cur_dir + f'raw_data/{run}/test_labels_chunk_classifier.npy')
test_labels_2 = np.load(cur_dir + f'raw_data/{run}/test_labels_chunk.npy')

# combine training data and shuffle and save as one
test_chunks = np.concatenate((test_chunks_1,test_chunks_2),axis=0)
test_labels = np.concatenate((test_labels_1,np.ones(test_labels_1.shape[0]).reshape(test_labels_1.shape[0],1)),axis=0)

# shuffle
np.random.seed(42)

indices = np.arange(test_chunks.shape[0])
np.random.shuffle(indices)

test_chunks = test_chunks[indices,:]
test_labels = test_labels[indices,:]

np.save(cur_dir + f'raw_data/{run}/test_data_classifier.npy',test_chunks)
np.save(cur_dir + f'raw_data/{run}/test_labels_classifier.npy',test_labels)

In [30]:
val_chunks_1 = np.load(cur_dir + f'raw_data/{run}/val_data_chunk_classifier.npy')
val_chunks_2 = np.load(cur_dir + f'raw_data/{run}/val_data_chunk.npy')
val_labels_1 = np.load(cur_dir + f'raw_data/{run}/val_labels_chunk_classifier.npy')
val_labels_2 = np.load(cur_dir + f'raw_data/{run}/val_labels_chunk.npy')

# combine training data and shuffle and save as one
val_chunks = np.concatenate((val_chunks_1,val_chunks_2),axis=0)
val_labels = np.concatenate((val_labels_1,np.ones(val_labels_1.shape[0]).reshape(val_labels_1.shape[0],1)),axis=0)

# shuffle
np.random.seed(42)

indices = np.arange(val_chunks.shape[0])
np.random.shuffle(indices)

val_chunks = val_chunks[indices,:]
val_labels = val_labels[indices,:]

np.save(cur_dir + f'raw_data/{run}/val_data_classifier.npy',val_chunks)
np.save(cur_dir + f'raw_data/{run}/val_labels_classifier.npy',val_labels)